In [1]:
'''
Pick bottom water calcite saturation field 'carb_ohm_cal_ben' using user-defined bathymetry
Update cGENIE output file of sedgem fields_sedgem_2d.nc
DeepDA using Baymag PSM may require the target field of "carb_ohm_cal_ben".
cGENIE output carb_ben_ohm_cal in fields_biogem_2d.nc file 
    corresponds to benthic ohmega for the base config topo, not user-defined bathymetry, thus is not useful.

By Mingsong Li
Date: April 23, 2020

'''

import yaml
import numpy as np
import pandas
import os
from netCDF4 import Dataset
from shutil import copyfile

sedgem_nc = 'fields_sedgem_2d.nc'
nc_field = 'carb_ohm_cal_ben'
biogem_nc = 'fields_biogem_3d.nc'
nc_field_ref = 'carb_ohm_cal'
t=0
dum_imax = 36
dum_jmax = 36
dum_dmax = 16
config_dir = '/mnt/c/Users/mul450/Dropbox/git/deepDA/'
config_name = "DeepDA_config.yml"
#config_name = "petmproxy3slices_v0.0.10gt1.csvexp_petm78_og1_qc_obs_20200203_test2.yml"
f = open(config_dir+config_name, 'r')
yml_dict = yaml.load(f, Loader=yaml.FullLoader)
f.close()

In [2]:
# *********************************************************************** %
def get_gdep(kmax, ez0, maxdep):
    z1 = ez0*(pow((1.0 + 1/ez0),(1.0/kmax)) - 1.0)
    tv4 = ez0*(pow((z1/ez0+1),0.5)-1)
    tv2 = 0
    tv1 = 0
    zro = np.zeros((kmax,1))
    zw = np.zeros((kmax+1,1))
    dz = np.zeros((kmax,1))
    dza = np.zeros((kmax,1))
    zro[kmax-1] = -tv4
    zw[kmax-1] = tv2

    for k in np.arange(1,kmax):
        if ez0 > 0:
            tv3 = ez0*(pow((z1/ez0+1),k)-1)
            dz[kmax-k] = tv3 - tv2
            tv2 = tv3
            tv5 = ez0*(pow((z1/ez0+1),(k+0.5))-1)
            if k < kmax:
                dza[kmax-k-1] = tv5 - tv4
            tv4 = tv5
            tv1 = tv1 + dz[kmax-k]
        else:
            dz[k-1] = 1.0/kmax
            dza[k-1] = 1.0/kmax

    for k in np.arange(kmax,1,-1):
        if k > 1:
            zro[k-2] = zro[k-1] - dza[k-2]
        zw[k-1] = zw[k] - dz[k-1]
    a = zw * maxdep
    a[0] = -1*maxdep # ensure the deepest one is 5km
    return a

# *********************************************************************** %
def calc_find_k(dum_depth, dum_kmax):
    par_D_max = 5000.0          # max depth (m)
    par_ez0 = 0.1               # GOLDSTEIN ez0 parameter
# set depth array
    D = get_gdep(dum_kmax,par_ez0,par_D_max) * -1
    #print(D)
    # set max k as default value - surface ocean
    loc_n_k = dum_kmax
    for n_k in np.arange(dum_kmax,2,-1):
        if dum_depth > D[n_k-1]:
            loc_n_k = n_k - 1
    if dum_depth > D[0]:
        loc_n_k = 1
    dum_k = [loc_n_k]
    return dum_k

In [3]:
dir_prior = yml_dict['core']['prior_dir']
nens = yml_dict['core']['nens']
dir_prior_full = os.listdir(dir_prior)
print(dir_prior_full[0])
prior_len = len(dir_prior_full)

print('>>    Read user-defined bathymetry, find bottom water calcite saturation')

for i in range(prior_len):
    name_nc_3d = dir_prior+'/'+ dir_prior_full[i]+'/'+ 'biogem' + '/' + biogem_nc
    ohm_cal_i = Dataset(name_nc_3d).variables[nc_field_ref][t,:,:,:]

    name_sedgem_2d = dir_prior+'/'+dir_prior_full[i]+'/sedgem/'+ sedgem_nc
    waterdepth = Dataset(name_sedgem_2d).variables['grid_topo']

    x = np.full((dum_imax, dum_jmax),np.nan)
    # step 1: find user-defined bathy   # step 2: find depth id k for each grid # step 3: read carb_ohm_cal, save as x

    for dum_imax_i in range(dum_imax):
        for dum_jmax_i in range(dum_jmax):
            
            waterdepthi = -1 * waterdepth[dum_imax_i, dum_jmax_i]
            ki = calc_find_k(waterdepthi, dum_dmax)
            kii = 16-ki[0]
            
            ohm_cal_val = np.ma.getdata(ohm_cal_i[kii,dum_imax_i,dum_jmax_i])
            ohm_cal_sur = np.ma.getdata(ohm_cal_i[0,dum_imax_i,dum_jmax_i])
            while ohm_cal_val<=0 and ohm_cal_sur>0:                
                kii = kii - 1
                #print('{}'.format(kii))
                ohm_cal_val = np.ma.getdata(ohm_cal_i[kii,dum_imax_i,dum_jmax_i])
                #print(ohm_cal_val)
                
            x[dum_imax_i,dum_jmax_i] = ohm_cal_val
            #print('i {} j {} depth {} ki {} kii {} ohm {} ohm_sur {}'.format(dum_imax_i, dum_jmax_i, waterdepthi, 16-ki[0], kii, ohm_cal_val, ohm_cal_i[0,dum_imax_i,dum_jmax_i]))
    x = x.reshape(1,36,36)
    x = np.ma.MaskedArray(x, x == 0.0)
    #print(x)
    nf = Dataset(name_sedgem_2d, 'a', format='NETCDF4')
    carb_ohm_cal = nf.createVariable(nc_field, 'f4', ('time','lat', 'lon'))
    carb_ohm_cal[:,:,:] = x
    carb_ohm_cal.units = ''
    nf.close()  # close the new file
    print('√ √   {} / {} Append 2d file: {}'.format(i+1,nens,name_sedgem_2d))
print('>>  End writing netCDF')

ML.petm009.ID.1
>>    Read user-defined bathymetry, find bottom water calcite saturation
√ √   0 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.1/sedgem/fields_sedgem_2d.nc
√ √   1 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.10/sedgem/fields_sedgem_2d.nc
√ √   2 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.100/sedgem/fields_sedgem_2d.nc
√ √   3 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.101/sedgem/fields_sedgem_2d.nc
√ √   4 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.102/sedgem/fields_sedgem_2d.nc
√ √   5 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.103/sedgem/fields_sedgem_2d.nc
√ √   6 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.104/sedgem/fields_sedgem_2d.nc
√ √   7 / 200 Append 2d file: /mnt/d/cGENIE/ML.petm/ML.petm009/ML.petm009.ID.105/sedgem/fields_sedgem_2d.nc
√ √   8 / 200 Append 2d file: /mnt/d/cGENIE/ML.pet

In [6]:
nf.close()  # close the new file